## 🎯 Here’s the Cool Part  

Since each of the 1‑200 **Christmas trees** in this competition is independent 🎄, we’ve brought together **ALL** available open‑source solutions 🤝 — merging them into **one super‑strong final answer**!  

The code is designed for **flexibility** 🧩:  
- 🔄 **Update results** anytime  
- 📓 **Drop in new open‑source notebooks**  
- 🚀 **Push toward the top score** with your own improvements



📝 How to Use & Update
To get the best results, don’t forget to:

🔄 Update the public notebooks already added

📥 Manually add any new open-source notebooks that become available

🎯 Quick Tip
Remember to refresh existing notebooks or drop in new ones to keep your solution up to date!

✨ Stay Updated
Keep your solution strong by:

🔄 Updating added notebooks

➕ Adding new open-source notebooks as they’re released



In [1]:
#!pip install shapely numba

In [2]:
import shutil


shutil.copy('/kaggle/input/why-not/submission.csv',  '/kaggle/working/submission1.csv')
shutil.copy('/kaggle/input/santa-submission/submission.csv',  '/kaggle/working/submission2.csv')
# shutil.copy('/kaggle/input/santa2025-sa-optimal-claus/submission.csv',  '/kaggle/working/submission3.csv')
shutil.copy('/kaggle/input/new-year-same-old-bbox/submission.csv',  '/kaggle/working/submission4.csv')
shutil.copy('/kaggle/input/magic-fast-backprop/submission.csv',  '/kaggle/working/submission5.csv')
shutil.copy('/kaggle/input/fork-of-santa-submission-happy-new-year/submission.csv',  '/kaggle/working/submission6.csv')
shutil.copy('/kaggle/input/a-bit-better-the-best-open-source-result-for-date/submission.csv',  '/kaggle/working/submission7.csv')
shutil.copy('/kaggle/input/k/datafad/new-year-same-old-bbox/submission.csv',  '/kaggle/working/submission8.csv')
shutil.copy('/kaggle/input/sa-on-metamodel-from-2-blocks-88-trees/results.csv',  '/kaggle/working/submission9.csv')
shutil.copy('/kaggle/input/squeezing-bbox-solution/submission_best.csv',  '/kaggle/working/submission10.csv')

shutil.copy('/kaggle/input/sa-on-metamodel-from-2-blocks-132-trees/submission_best.csv',  '/kaggle/working/submission11.csv')
shutil.copy('/kaggle/input/a-super-simple-improvement-strategy/submission.csv',  '/kaggle/working/submission12.csv')

submission_files = [
        '/kaggle/working/submission1.csv',
        '/kaggle/working/submission2.csv',
        # '/kaggle/working/submission3.csv',
        '/kaggle/working/submission4.csv',
        '/kaggle/working/submission5.csv',
        '/kaggle/working/submission6.csv',
        '/kaggle/working/submission7.csv',
        '/kaggle/working/submission8.csv',
        '/kaggle/working/submission9.csv',
        '/kaggle/working/submission10.csv',
        '/kaggle/working/submission11.csv',
        '/kaggle/working/submission12.csv',
    
        
    ]

In [3]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree

# Set precision for Decimal (25 is good for contest standards)
getcontext().prec = 25
scale_factor = Decimal("1e18")


class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x="0", center_y="0", angle="0"):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)

        trunk_w = Decimal("0.15")
        trunk_h = Decimal("0.2")
        base_w = Decimal("0.7")
        mid_w = Decimal("0.4")
        top_w = Decimal("0.25")
        tip_y = Decimal("0.8")
        tier_1_y = Decimal("0.5")
        tier_2_y = Decimal("0.25")
        base_y = Decimal("0.0")
        trunk_bottom_y = -trunk_h

        # Define the 15 vertices of the tree polygon
        initial_polygon = Polygon(
            [
                (Decimal("0.0") * scale_factor, tip_y * scale_factor),
                (top_w / Decimal("2") * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal("4") * scale_factor, tier_1_y * scale_factor),
                (mid_w / Decimal("2") * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal("4") * scale_factor, tier_2_y * scale_factor),
                (base_w / Decimal("2") * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal("2") * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal("2") * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal("2")) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal("2")) * scale_factor, base_y * scale_factor),
                (-(base_w / Decimal("2")) * scale_factor, base_y * scale_factor),
                (-(mid_w / Decimal("4")) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal("2")) * scale_factor, tier_2_y * scale_factor),
                (-(top_w / Decimal("4")) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal("2")) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        
        # Apply rotation and translation to the polygon
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated, 
            xoff=float(self.center_x * scale_factor), 
            yoff=float(self.center_y * scale_factor)
        )


def load_configuration_from_df(n: int, df: pd.DataFrame) -> list[ChristmasTree]:
    """
    Loads all trees for a given N from the submission DataFrame.
    """
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        # Remove 's' prefix and convert to string for Decimal constructor
        x = str(row["x"])[1:]
        y = str(row["y"])[1:]
        deg = str(row["deg"])[1:]
        
        # Ensure values are present before passing to ChristmasTree constructor
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
        else:
             # Handle cases where configuration might be incomplete/missing
             pass 
             
    return trees


def get_score(trees: list[ChristmasTree], n: int) -> float:
    """
    Calculates the score (S^2 / N) for a given configuration of trees.
    S is the side length of the minimum bounding square.
    """
    if not trees:
        return 0.0

    # Collect all exterior points from all tree polygons, scale them back down
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / float(scale_factor) for t in trees])
    
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    
    side_length = max(max_x - min_x, max_y - min_y)
    
    # Score is S^2 / N
    score = side_length**2 / n
    return score

def has_overlap(trees: list[ChristmasTree]) -> bool:
    """Check if any two ChristmasTree polygons overlap."""
    if len(trees) <= 1:
        return False

    polygons = [t.polygon for t in trees]
    # Use STRtree for efficient proximity queries (optimizes checking pairs)
    tree_index = STRtree(polygons)

    for i, poly in enumerate(polygons):
        # Query for polygons whose bounding boxes overlap with poly
        # This returns the indices of potential overlaps
        indices = tree_index.query(poly)
        
        for idx in indices:
            # Skip checking the polygon against itself
            if idx == i:
                continue
                
            # Perform the precise intersection check
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                # Overlap found!
                return True
    return False

# ----------------------------------------------------------------------

def score_and_validate_submission(file_path: str, max_n: int = 200) -> dict:
    """
    Reads a submission CSV, calculates the total score, and checks for overlaps 
    in all configurations (N=1 up to max_n).
    """
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return {"status": "FAILED", "error": "File Not Found"}
    except Exception as e:
        print(f"Error reading CSV: {e}")
        return {"status": "FAILED", "error": f"CSV Read Error: {e}"}

    total_score = 0.0
    failed_overlap_n = []
    
    print(f"--- Scoring and Validation: {file_path} (N=1 to {max_n}) ---")

    for n in range(1, max_n + 1):
        trees = load_configuration_from_df(n, df)
        
        # Only process if there are trees for this N in the file
        if trees:
            current_score = get_score(trees, n)
            total_score += current_score

            if has_overlap(trees):
                failed_overlap_n.append(n)
                print(f"  ❌ N={n:03d}: OVERLAP DETECTED! (Score contribution: {current_score:.6f})")
            else:
                # Optionally print success for each N
                # print(f"  ✅ N={n:03d}: OK (Score contribution: {current_score:.6f})")
                pass
        
    print("\n--- Summary ---")
    if failed_overlap_n:
        print(f"❌ **Validation FAILED**: Overlaps found in N: {failed_overlap_n}")
        status = "FAILED (Overlaps)"
    else:
        print("✅ **Validation SUCCESSFUL**: No overlaps detected.")
        status = "SUCCESS"
        
    print(f"**Total Submission Score (Σ S²/N): {total_score:.6f}**")
    
    return {
        "status": status,
        "total_score": total_score,
        "failed_overlap_n": failed_overlap_n
    }



In [4]:
from pathlib import Path

def merge_best_submissions(file_paths: list[str], output_path: str, max_n: int = 200) -> dict:
    """
    Merges multiple submission files by selecting the best (lowest score) configuration for each N.
    
    Args:
        file_paths: List of paths to submission CSV files
        output_path: Path to save the merged best submission
        max_n: Maximum N value to process
    
    Returns:
        Dictionary with merge statistics and validation results
    """
    print(f"--- Merging {len(file_paths)} submission files ---")
    
    # Load all dataframes
    dfs = []
    for fp in file_paths:
        try:
            df = pd.read_csv(fp)
            dfs.append((fp, df))
            print(f"✓ Loaded: {fp} ({len(df)} rows)")
        except Exception as e:
            print(f"✗ Failed to load {fp}: {e}")
    
    if not dfs:
        return {"status": "FAILED", "error": "No valid files loaded"}
    
    # Store best configurations for each N
    best_configs = []
    selection_stats = {}
    
    for n in range(1, max_n + 1):
        best_score = float('inf')
        best_trees_data = None
        best_source = None
        
        # Compare configurations from all files for this N
        for file_path, df in dfs:
            trees = load_configuration_from_df(n, df)
            
            if trees:
                score = get_score(trees, n)
                
                # Check if this is the best score so far
                if score < best_score:
                    best_score = score
                    # Extract the rows for this N from the dataframe
                    best_trees_data = df[df["id"].str.startswith(f"{n:03d}_")].copy()
                    best_source = Path(file_path).name
        
        # Add the best configuration to the result
        if best_trees_data is not None:
            best_configs.append(best_trees_data)
            selection_stats[n] = {
                "score": best_score,
                "source": best_source
            }
            print(f"N={n:03d}: Best score={best_score:.6f} from {best_source}")
    
    # Combine all best configurations
    if not best_configs:
        return {"status": "FAILED", "error": "No valid configurations found"}
    
    merged_df = pd.concat(best_configs, ignore_index=True)
    
    # Save merged result
    merged_df.to_csv(output_path, index=False)
    print(f"\n✓ Merged submission saved to: {output_path}")
    
    # Validate the merged result
    print("\n--- Validating Merged Submission ---")
    validation_result = score_and_validate_submission(output_path, max_n=max_n)
    
    # Add selection statistics
    source_counts = {}
    for stats in selection_stats.values():
        source = stats["source"]
        source_counts[source] = source_counts.get(source, 0) + 1
    
    print("\n--- Source File Usage ---")
    for source, count in sorted(source_counts.items(), key=lambda x: -x[1]):
        print(f"  {source}: {count} configurations ({count/max_n*100:.1f}%)")
    
    return {
        "status": validation_result["status"],
        "total_score": validation_result["total_score"],
        "failed_overlap_n": validation_result.get("failed_overlap_n", []),
        "merged_file": output_path,
        "source_counts": source_counts,
        "selection_stats": selection_stats
    }


# Example usage
if __name__ == "__main__":
    # List your submission files here
    
    output_file = '/kaggle/working/submission.csv'
    
    result = merge_best_submissions(
        file_paths=submission_files,
        output_path=output_file,
        max_n=200
    )
    
    print("\n=== Final Result ===")
    print(result)

--- Merging 11 submission files ---
✓ Loaded: /kaggle/working/submission1.csv (20100 rows)
✓ Loaded: /kaggle/working/submission2.csv (20100 rows)
✓ Loaded: /kaggle/working/submission4.csv (20100 rows)
✓ Loaded: /kaggle/working/submission5.csv (20100 rows)
✓ Loaded: /kaggle/working/submission6.csv (20100 rows)
✓ Loaded: /kaggle/working/submission7.csv (20100 rows)
✓ Loaded: /kaggle/working/submission8.csv (20100 rows)
✓ Loaded: /kaggle/working/submission9.csv (20100 rows)
✓ Loaded: /kaggle/working/submission10.csv (20100 rows)
✓ Loaded: /kaggle/working/submission11.csv (20100 rows)
✓ Loaded: /kaggle/working/submission12.csv (20100 rows)
N=001: Best score=0.661250 from submission1.csv
N=002: Best score=0.450779 from submission1.csv
N=003: Best score=0.434745 from submission1.csv
N=004: Best score=0.416545 from submission1.csv
N=005: Best score=0.416850 from submission1.csv
N=006: Best score=0.399610 from submission1.csv
N=007: Best score=0.399897 from submission1.csv
N=008: Best score=0.